In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


In [2]:
data = pd.read_csv("C:\loan fraud\data\Default_Fin.csv")

print("Dataset information: ")
print(data.info())

print(data.head())

Dataset information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Index          10000 non-null  int64  
 1   Employed       10000 non-null  int64  
 2   Bank Balance   10000 non-null  float64
 3   Annual Salary  10000 non-null  float64
 4   Defaulted?     10000 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 390.8 KB
None
   Index  Employed  Bank Balance  Annual Salary  Defaulted?
0      1         1       8754.36      532339.56           0
1      2         0       9806.16      145273.56           0
2      3         1      12882.60      381205.68           0
3      4         1       6351.00      428453.88           0
4      5         1       9427.92      461562.00           0


<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
C:\Users\HP\AppData\Local\Temp\ipykernel_15736\3905365063.py:1: SyntaxWarning: invalid escape sequence '\l'
  data = pd.read_csv("C:\loan fraud\data\Default_Fin.csv")


In [3]:
print("Total Missing Values:")
print(data.isnull().sum())


data.fillna(method='bfill', inplace=True)


print("Missing values count after modification: ")
print(data.isnull().sum())

Total Missing Values:
Index            0
Employed         0
Bank Balance     0
Annual Salary    0
Defaulted?       0
dtype: int64
Missing values count after modification: 
Index            0
Employed         0
Bank Balance     0
Annual Salary    0
Defaulted?       0
dtype: int64


C:\Users\HP\AppData\Local\Temp\ipykernel_15736\3664967689.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)


In [4]:
categorical_columns =  data.select_dtypes(include=['object']).columns
print("Categorical Columns: ", categorical_columns)

label_enc = LabelEncoder()
for col in categorical_columns:
    data[col] = label_enc.fit_trasform(data[col])
print("Data after encoding: ")
print(data.head())


Categorical Columns:  Index([], dtype='object')
Data after encoding: 
   Index  Employed  Bank Balance  Annual Salary  Defaulted?
0      1         1       8754.36      532339.56           0
1      2         0       9806.16      145273.56           0
2      3         1      12882.60      381205.68           0
3      4         1       6351.00      428453.88           0
4      5         1       9427.92      461562.00           0


In [5]:
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns

scaler = StandardScaler()
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

print("Data after scaling:")
print(data.head())

Data after scaling:
      Index  Employed  Bank Balance  Annual Salary  Defaulted?
0 -1.731878  0.645936     -0.218828       0.813188   -0.185599
1 -1.731531 -1.548141     -0.037617      -1.605496   -0.185599
2 -1.731185  0.645936      0.492412      -0.131212   -0.185599
3 -1.730838  0.645936     -0.632894       0.164031   -0.185599
4 -1.730492  0.645936     -0.102782       0.370915   -0.185599


In [6]:
from sklearn.model_selection import train_test_split

X = data.drop('Defaulted?', axis = 1)
y = data['Defaulted?']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)


print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

Training set size: (8000, 4)
Test set size: (2000, 4)


In [7]:
X_train.to_csv('C:/loan fraud/data/X_train.csv', index=False)
X_test.to_csv('C:/loan fraud/data/X_test.csv', index=False)
y_train.to_csv('C:/loan fraud/data/y_train.csv', index=False)
y_test.to_csv('C:/loan fraud/data/y_test.csv', index=False)

# Convert to binary values
data['Defaulted?'] = data['Defaulted?'].apply(lambda x: 1 if x == 'Yes' else 0)


from sklearn.model_selection import train_test_split

X = data.drop('Defaulted?', axis = 1)
y = data['Defaulted?']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)


print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

xgb_model = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=6)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)
print("XGBoost Accuracy: ", accuracy_score(y_test, y_pred_xgb))
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb))
print("XGBoost Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))

joblib.dump(xgb_model, 'fraud_detection_xgb_model.pkl')


Training set size: (8000, 4)
Test set size: (2000, 4)
XGBoost Accuracy:  1.0
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

XGBoost Confusion Matrix:
 [[2000]]


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


['fraud_detection_xgb_model.pkl']

In [8]:
print(y.head())

y = (y > 0).astype(int)

X = data.drop('Defaulted?', axis=1)  # Features
y = (data['Defaulted?'] > 0).astype(int)  # Target variable, now binary

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=6)

xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print("XGBoost Accuracy: ", accuracy_score(y_test, y_pred_xgb))
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb))
print("XGBoost Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))

0    0
1    0
2    0
3    0
4    0
Name: Defaulted?, dtype: int64
XGBoost Accuracy:  1.0
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

XGBoost Confusion Matrix:
 [[2000]]


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:

y_train = y_train.astype(int)  # Ensure integer type if needed
y_test = y_test.astype(int)


In [10]:
print("Full dataset target distribution:", y.value_counts())
print("Training set target distribution:", y_train.value_counts())
print("Test set target distribution:", y_test.value_counts())

Full dataset target distribution: Defaulted?
0    10000
Name: count, dtype: int64
Training set target distribution: Defaulted?
0    8000
Name: count, dtype: int64
Test set target distribution: Defaulted?
0    2000
Name: count, dtype: int64


In [10]:
print(y.value_counts())


Defaulted?
0    10000
Name: count, dtype: int64


In [12]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

X = data.drop('Defaulted?', axis=1)
y = data['Defaulted?'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


lgb_model = lgb.LGBMClassifier(n_estimators=1000, learning_rate=0.1, max_depth=6, objective='binary')
lgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='logloss', callbacks=[lgb.early_stopping(100)])

y_pred = lgb_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 0, number of negative: 8000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 768
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 4
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000000 -> initscore=-34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] Stopped training because there are no more leaves that m